In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd

In [18]:
cases = 'CasesData.csv'
cases_df =  pd.read_csv(cases)
cases_df.head()

,Name,2008_Cases,2009_Cases,2010_Cases,2011_Cases,2012_Cases,2013_Cases,2014_Cases,2015_Cases,2016_Cases,2017_Cases,2018_Cases
0,Alabama,709,695,681,683,666,635,669,663,663,651,572
1,Alaska,39,21,35,24,28,23,38,25,37,34,20
2,Arizona,679,642,619,563,620,689,741,690,714,725,806
3,Arkansas,238,280,234,241,247,262,312,273,312,287,281
4,California,5719,5417,5272,5044,5050,4711,5169,5108,5172,4766,4398


In [3]:
years = ["2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]
urls = ['https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2008-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2009-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2010-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2011-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2013-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2014-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2015-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2016-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2017-01-01#','https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2018-01-01#']

In [19]:
for year in years:
    if year != "2018":
        url = f'https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od={year}-01-01#'
        print(url)
        tables = pd.read_html(url)
        pop_df = pd.DataFrame(tables[0])
        pop_df = pop_df.iloc[:, [1,2]]
        pop_df.columns = pop_df.columns.droplevel(0)
        pop_df.drop(pop_df[pop_df['Name'] == "FRB-St. Louis District States"].index, inplace=True)
        pop_df[year] *= 1000
        cases_df = pd.merge(cases_df, pop_df, how='left', on='Name', sort=False)
        cases_df[f"{year} HIV Rate"] = cases_df[f"{year}_Cases"] / cases_df[year]
        cases_df.drop([f"{year}_Cases", year], axis=1, inplace=True)
    else:
        url = f'https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od={year}-01-01#'
        print(url)
        tables = pd.read_html(url)
        pop_df = pd.DataFrame(tables[0])
        pop_df = pop_df.iloc[:, [1,3]]
        pop_df.rename(columns={"Value": "2018"}, inplace=True)
        pop_df.columns = pop_df.columns.droplevel(0)
        pop_df.drop(pop_df[pop_df['Name'] == "FRB-St. Louis District States"].index, inplace=True)
        pop_df["2018"] = pop_df["2018"].astype(float)
        pop_df["Name"] = pop_df["Name"].astype(str)
        pop_df["2018"] *= 1000
        cases_df = pd.merge(cases_df, pop_df, how='left', on='Name', sort=False)
        cases_df[f"{year} HIV Rate"] = cases_df[f"{year}_Cases"] / cases_df[year]
        cases_df.drop([f"{year}_Cases", year], axis=1, inplace=True)        

https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2008-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2009-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2010-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2011-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2012-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2013-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2014-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2015-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2016-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2017-01-01#
https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2018-01-01#


In [20]:
cases_df.head()
#print(pop_df)

,Name,2008 HIV Rate,2009 HIV Rate,2010 HIV Rate,2011 HIV Rate,2012 HIV Rate,2013 HIV Rate,2014 HIV Rate,2015 HIV Rate,2016 HIV Rate,2017 HIV Rate,2018 HIV Rate
0,Alabama,0.000150,0.000146,0.000142,0.000142,0.000138,0.000131,0.000138,0.000137,0.000136,0.000134,0.000117
1,Alaska,0.000057,0.000030,0.000049,0.000033,0.000038,0.000031,0.000052,0.000034,0.000050,0.000046,0.000027
2,Arizona,0.000108,0.000101,0.000097,0.000087,0.000095,0.000104,0.000110,0.000101,0.000103,0.000103,0.000113
3,Arkansas,0.000083,0.000097,0.000080,0.000082,0.000084,0.000089,0.000105,0.000092,0.000104,0.000096,0.000093
4,California,0.000156,0.000147,0.000141,0.000134,0.000133,0.000123,0.000134,0.000131,0.000132,0.000121,0.000111


In [21]:
cases_df['New HIV Case Rate 2008-2012'] = cases_df[['2008 HIV Rate','2009 HIV Rate','2010 HIV Rate','2011 HIV Rate','2012 HIV Rate']].mean(axis=1)
cases_df['New HIV Case Rate 2013-2018'] = cases_df[['2013 HIV Rate','2014 HIV Rate','2015 HIV Rate','2016 HIV Rate','2017 HIV Rate','2018 HIV Rate']].mean(axis=1)
cases_df = cases_df.drop(['2008 HIV Rate','2009 HIV Rate','2010 HIV Rate','2011 HIV Rate','2012 HIV Rate','2013 HIV Rate','2014 HIV Rate','2015 HIV Rate','2016 HIV Rate','2017 HIV Rate','2018 HIV Rate'], axis=1)
cases_df.head()

,Name,New HIV Case Rate 2008-2012,New HIV Case Rate 2013-2018
0,Alabama,0.000144,0.000132
1,Alaska,0.000041,0.000040
2,Arizona,0.000098,0.000106
3,Arkansas,0.000085,0.000096
4,California,0.000142,0.000125


In [22]:
cases_df["New HIV Case Rate Variance"] = cases_df["New HIV Case Rate 2008-2012"] - cases_df["New HIV Case Rate 2013-2018"]
cases_df.head()

,Name,New HIV Case Rate 2008-2012,New HIV Case Rate 2013-2018,New HIV Case Rate Variance
0,Alabama,0.000144,0.000132,0.000012
1,Alaska,0.000041,0.000040,0.000002
2,Arizona,0.000098,0.000106,-0.000008
3,Arkansas,0.000085,0.000096,-0.000011
4,California,0.000142,0.000125,0.000017


In [8]:
cases_df.dtypes

Name                            object
New HIV Case Rate 2008-2012    float64
New HIV Case Rate 2013-2018    float64
New HIV Case Rate Variance     float64
dtype: object

In [23]:
cases_df['New HIV Case Rate 2008-2012'] *= 100000
cases_df['New HIV Case Rate 2013-2018'] *= 100000
cases_df['New HIV Case Rate Variance'] *= 100000

# new_case_df = case_df.loc['New HIV Case Rate 2008-2012', 'New HIV Case Rate 2013-2018', 'New HIV Case Rate Variance'] *= 100000
# new_cases_df = cases_df.select_dtypes(include=['float64']).mul(100000)
# new_cases_df = cases_df.mul(100000)

cases_df.head()

,Name,New HIV Case Rate 2008-2012,New HIV Case Rate 2013-2018,New HIV Case Rate Variance
0,Alabama,14.385351,13.219613,1.165738
1,Alaska,4.147444,3.996431,0.151014
2,Arizona,9.750057,10.556607,-0.806551
3,Arkansas,8.503143,9.644722,-1.141579
4,California,14.223003,12.548216,1.674786


In [24]:
cases_df.to_csv("New_HIV_Case_Rates.csv", index=False)